In [15]:
# Import modules
from logger import setup_custom_logger
import sys
import pandas as pd    
import cbsodata
import datetime

In [16]:
# Setup of logger
try:
    logger = setup_custom_logger("E001_BBP")
    logger.info('starting')
except:
    logger.exception('logger could not be loaded')
    raise

2021-03-19 13:00:18 INFO     E001_BBP starting
2021-03-19 13:00:18 INFO     E001_BBP starting
2021-03-19 13:00:18 INFO     E001_BBP starting


INFO:E001_BBP:starting


In [17]:
try:
    # Get current date information
    now = datetime.datetime.now()
    
    logger.info('datetime loaded')
    yearMin = now.year-4
    yearMax = now.year
    
    # Values you can load
    # now.year
    # now.month
    # now.day
    # now.hour
    # now.minute
except:
    logger.exception('datetime could not be loaded')
    logger.info('set yearmin to a default value')
    # Set default values for fallback
    yearMin = 2010
    yearMax = 2030
    raise

2021-03-19 13:00:18 INFO     E001_BBP datetime loaded
2021-03-19 13:00:18 INFO     E001_BBP datetime loaded
2021-03-19 13:00:18 INFO     E001_BBP datetime loaded


INFO:E001_BBP:datetime loaded


In [18]:
# Dataset 84106NED
dataset_id = '84106NED'

# Table definitions

# SoortMutaties: A045299 = Volume, t.o.v. zelfde periode vorig jaar
# BrutoBinnenlandsProduct_2: waarde BBP

In [19]:
try:
    logger.info(f'Retrieve data from dataset {dataset_id}')
    data = pd.DataFrame(cbsodata.get_data(
    f'{dataset_id}',
    filters=f"substring(Perioden,0,4) ge '{yearMin}' and SoortMutaties eq 'A045299'",
    select=["Perioden", "BrutoBinnenlandsProduct_2"]))
except:
    logger.exception('error loading data from CBS Statline')
    raise

2021-03-19 13:00:27 INFO     E001_BBP Retrieve data from dataset 84106NED
2021-03-19 13:00:27 INFO     E001_BBP Retrieve data from dataset 84106NED
2021-03-19 13:00:27 INFO     E001_BBP Retrieve data from dataset 84106NED


INFO:E001_BBP:Retrieve data from dataset 84106NED


In [20]:
# Remove quaterly and yearly data
try:
    data = data[data["Perioden"].str.contains("kwartaal")==True]
    data = data[data['Perioden'].map(len) > 4 ]
except:
    logger.exception('Perioden filter could not be applied')
    raise

In [21]:
# Rename columns
try:
    data = data.rename(columns={
    "BrutoBinnenlandsProduct_2": "Volume_tovv_zelfde_periode_vorig_jaar_in_procenten",
})

except:
    logger.exeption('Columns could not be renamed')
    raise

In [22]:
# Date formatting and quarter format
try:
    data["Perioden"] = data["Perioden"].str.replace(" 1e kwartaal", "-03-01")
    data["Perioden"] = data["Perioden"].str.replace(" 2e kwartaal", "-06-01")
    data["Perioden"] = data["Perioden"].str.replace(" 3e kwartaal", "-09-01")
    data["Perioden"] = data["Perioden"].str.replace(" 4e kwartaal", "-12-01")
    data["Perioden"] = pd.to_datetime(data["Perioden"], format='%Y-%m-%d')
    data['Perioden'] = pd.to_datetime(data["Perioden"]).dt.date)
    data
except:
    logger.exception('Columns could not be changed to monthly numbers or formatted to different date')
    raise

2021-03-23 10:05:53 ERROR    E001_BBP Columns could not be changed to monthly numbers or formatted to different date
Traceback (most recent call last):
  File "<ipython-input-18-c0f9a51300ac>", line 3, in <module>
    data["Perioden"] = data["Perioden"].str.replace(" 1e kwartaal", "-03-01")
  File "D:\Software\Anaconda\lib\site-packages\pandas\core\generic.py", line 5135, in __getattr__
    return object.__getattribute__(self, name)
  File "D:\Software\Anaconda\lib\site-packages\pandas\core\accessor.py", line 187, in __get__
    accessor_obj = self._accessor(obj)
  File "D:\Software\Anaconda\lib\site-packages\pandas\core\strings.py", line 2100, in __init__
    self._inferred_dtype = self._validate(data)
  File "D:\Software\Anaconda\lib\site-packages\pandas\core\strings.py", line 2157, in _validate
    raise AttributeError("Can only use .str accessor with string values!")
AttributeError: Can only use .str accessor with string values!
2021-03-23 10:05:53 ERROR    E001_BBP Columns could n

ERROR:E001_BBP:Columns could not be changed to monthly numbers or formatted to different date
Traceback (most recent call last):
  File "<ipython-input-18-c0f9a51300ac>", line 3, in <module>
    data["Perioden"] = data["Perioden"].str.replace(" 1e kwartaal", "-03-01")
  File "D:\Software\Anaconda\lib\site-packages\pandas\core\generic.py", line 5135, in __getattr__
    return object.__getattribute__(self, name)
  File "D:\Software\Anaconda\lib\site-packages\pandas\core\accessor.py", line 187, in __get__
    accessor_obj = self._accessor(obj)
  File "D:\Software\Anaconda\lib\site-packages\pandas\core\strings.py", line 2100, in __init__
    self._inferred_dtype = self._validate(data)
  File "D:\Software\Anaconda\lib\site-packages\pandas\core\strings.py", line 2157, in _validate
    raise AttributeError("Can only use .str accessor with string values!")
AttributeError: Can only use .str accessor with string values!


AttributeError: Can only use .str accessor with string values!

In [ ]:
# Export dataFrame to Excel file
try:
    data.to_excel("E001_BBP.xlsx")  
except:
    logger.exception('dataFrame could not be exported to output folder')